# Single-impurity Anderson model

Yi-Te Huang  
2025-01-14

## Introduction

The investigation of the Kondo effect in single-impurity Anderson model
is crucial as it serves both as a valuable testing ground for the
theories of the Kondo effect and has the potential to lead to a better
understanding of this intrinsic many-body phenomena. For further
detailed discussions of this model (under different parameters) using
[`HierarchicalEOM.jl`](https://github.com/qutip/HierarchicalEOM.jl), we
recommend to read the article (Huang et al. 2023).

## Hamiltonian

We consider a single-level electronic system \[which can be populated by
a spin-up ($\uparrow$) or spin-down ($\downarrow$) electron\] coupled to
a fermionic reservoir ($\textrm{f}$). The total Hamiltonian is given by
$H_{\textrm{T}}=H_\textrm{s}+H_\textrm{f}+H_\textrm{sf}$, where each
terms takes the form

$$
\begin{aligned}
H_{\textrm{s}}  &= \epsilon \left(d^\dagger_\uparrow d_\uparrow + d^\dagger_\downarrow d_\downarrow \right) + U\left(d^\dagger_\uparrow d_\uparrow d^\dagger_\downarrow d_\downarrow\right),\\
H_{\textrm{f}}  &=\sum_{\sigma=\uparrow,\downarrow}\sum_{k}\epsilon_{\sigma,k}c_{\sigma,k}^{\dagger}c_{\sigma,k},\\
H_{\textrm{sf}} &=\sum_{\sigma=\uparrow,\downarrow}\sum_{k}g_{k}c_{\sigma,k}^{\dagger}d_{\sigma} + g_{k}^*d_{\sigma}^{\dagger}c_{\sigma,k}.
\end{aligned}
$$

Here, $d_\uparrow$ $(d_\downarrow)$ annihilates a spin-up (spin-down)
electron in the system, $\epsilon$ is the energy of the electron, and
$U$ is the Coulomb repulsion energy for double occupation. Furthermore,
$c_{\sigma,k}$ $(c_{\sigma,k}^{\dagger})$ annihilates (creates) an
electron in the state $k$ (with energy $\epsilon_{\sigma,k}$) of the
reservoir.

Now, we need to build the system Hamiltonian and initial state with the
package
[`QuantumToolbox.jl`](https://github.com/qutip/QuantumToolbox.jl) to
construct the operators.

In [1]:
using HierarchicalEOM  # this automatically loads `QuantumToolbox`
using CairoMakie       # for plotting results

In [1]:
ϵ = -5
U = 10
σm = sigmam() # σ-
σz = sigmaz() # σz
II = qeye(2)  # identity matrix

# construct the annihilation operator for both spin-up and spin-down
# (utilize Jordan–Wigner transformation)
d_up = tensor(σm, II)
d_dn = tensor(-1 * σz, σm)
Hsys = ϵ * (d_up' * d_up + d_dn' * d_dn) + U * (d_up' * d_up * d_dn' * d_dn)


Quantum Object:   type=Operator()   dims=[2, 2]   size=(4, 4)   ishermitian=true
4×4 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
     ⋅           ⋅           ⋅          ⋅    
     ⋅      -5.0+0.0im       ⋅          ⋅    
     ⋅           ⋅      -5.0+0.0im      ⋅    
     ⋅           ⋅           ⋅          ⋅    

## Construct bath objects

We assume the fermionic reservoir to have a [Lorentzian-shaped spectral
density](https://qutip.org/HierarchicalEOM.jl/stable/bath_fermion/Fermion_Lorentz/#doc-Fermion-Lorentz),
and we utilize the Padé decomposition. Furthermore, the spectral
densities depend on the following physical parameters:

-   the coupling strength $\Gamma$ between system and reservoirs
-   the band-width $W$
-   the product of the Boltzmann constant $k$ and the absolute
    temperature $T$ : $kT$
-   the chemical potential $\mu$
-   the total number of exponentials for the reservoir $2(N + 1)$

In [1]:
Γ = 2
μ = 0
W = 10
kT = 0.5
N = 5
bath_up = Fermion_Lorentz_Pade(d_up, Γ, μ, W, kT, N)
bath_dn = Fermion_Lorentz_Pade(d_dn, Γ, μ, W, kT, N)
bath_list = [bath_up, bath_dn]

2-element Vector{FermionBath}:
 HierarchicalEOM.FermionBath object with 12 terms.

 HierarchicalEOM.FermionBath object with 12 terms.

## Construct HEOMLS matrix

In [1]:
tier = 3
M_even = M_Fermion(Hsys, tier, bath_list)
M_odd = M_Fermion(Hsys, tier, bath_list, ODD)

Preparing block matrices for HEOM Liouvillian superoperator (using 4 threads)...
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Constructing matrix...[DONE]
Preparing block matrices for HEOM Liouvillian superoperator (using 4 threads)...
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)
Constructing matrix...[DONE]

Fermion type HEOMLS matrix acting on odd-parity ADOs
system dims = [2, 2]
number of ADOs N = 2325
data =
MatrixOperator(37200 × 37200)

## Solve stationary state of ADOs

In [1]:
ados_s = steadystate(M_even)

Solving steady state for ADOs by linear-solve method...
Calculating left preconditioner with ilu...[DONE]
Solving linear problem...[DONE]

2325 Auxiliary Density Operators with even-parity and (system) dims = [2, 2]

## Calculate density of states (DOS)

In [1]:
ωlist = -10:1:10
dos = DensityOfStates(M_odd, ados_s, d_up, ωlist)

Calculating density of states in frequency domain...
Progress: [=================             ]  57.1% --- Elapsed Time: 0h 00m 01s (ETA: 0h 00m 00s)Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 01s (ETA: 0h 00m 00s)
[DONE]

21-element Vector{Float64}:
 0.02434162065289487
 0.04465552042387327
 0.09077144051564014
 0.19298880556747186
 0.2935153689447
 0.23367107985907315
 0.15864674406911258
 0.12379877950506187
 0.11772111509217906
 0.145243037047314
 ⋮
 0.11772111509216472
 0.12379877950510276
 0.15864674406912144
 0.2336710798590303
 0.29351536894465224
 0.192988805567427
 0.0907714405156127
 0.04465552042384389
 0.024341620652875384

plot the results

In [1]:
fig = Figure(size = (500, 350))
ax = Axis(fig[1, 1], xlabel = L"\omega")
lines!(ax, ωlist, dos)

fig

## Version Information

In [1]:
HierarchicalEOM.versioninfo()

Huang, Yi-Te, Po-Chen Kuo, Neill Lambert, Mauro Cirio, Simon Cross,
Shen-Liang Yang, Franco Nori, and Yueh-Nan Chen. 2023. “An Efficient
Julia Framework for Hierarchical Equations of Motion in Open Quantum
Systems.” *Communications Physics* 6 (1): 313.
<https://doi.org/10.1038/s42005-023-01427-2>.